# 1. 自动识别文件

In [1]:
import os
import datetime

In [2]:
list_file = list(os.walk('./'))[0][2]
for file in list_file:
    if 'Lennon_整理好的替代料总表' in file:
        alternate = file
    elif '改价表' in file:
        stock = file
    elif 'US MRP' in file:
        mrp = file

print(f'alternate: \t{alternate}')
print(f'stock: \t\t{stock}')
print(f'mrp: \t\t{mrp}')

alternate: 	Lennon_整理好的替代料总表_20231024_161344.xlsx
stock: 		改价表 10-24.xlsx
mrp: 		US MRP20231023-运营.xlsx


# 2. 计算库存
## 2.1 替代料库存

In [3]:
import pandas as pd

In [4]:
df_alternate = pd.read_excel(alternate,
                             header=0).fillna('')
df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6
0,USCA171876D-3,USCA171876-3,USCA171876,USCA171876D,,,
1,USGS00429LR,USGS00429,USGS00429T,,,,
2,USWL02211,USWL02211H,,,,,
3,USTC35518,USETC35518D,USETC35518,,,,
4,USTHA7040P,USTHA7040PX,,,,,
...,...,...,...,...,...,...,...
44798,USGS01664-2,USGS01664-2-C,,,,,
44799,USASC06160-C,USASC06160,,,,,
44800,USASC16160-C,USASC16160,,,,,
44801,USASC20303-C,USASC20303,,,,,


In [5]:
df_alternate['row'] = [str(i) for i in range(len(df_alternate))]

df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,row
0,USCA171876D-3,USCA171876-3,USCA171876,USCA171876D,,,,0
1,USGS00429LR,USGS00429,USGS00429T,,,,,1
2,USWL02211,USWL02211H,,,,,,2
3,USTC35518,USETC35518D,USETC35518,,,,,3
4,USTHA7040P,USTHA7040PX,,,,,,4
...,...,...,...,...,...,...,...,...
44798,USGS01664-2,USGS01664-2-C,,,,,,44798
44799,USASC06160-C,USASC06160,,,,,,44799
44800,USASC16160-C,USASC16160,,,,,,44800
44801,USASC20303-C,USASC20303,,,,,,44801


In [6]:
df_stock = pd.read_excel(stock,
                         header=0,
                         names=['SKU', '当日库存'],
                         usecols='A, F')

df_stock

,SKU,当日库存
0,USBDA04239,0
1,USBDA04868,1
2,USDLA3ZP50,0
3,USEVC271FG,0
4,USBDA04007,0
...,...,...
269253,USIC11424I-8,1
269254,USICK2230I-8,1
269255,USOS44958-C,1
269256,USACC78398S-C,1


In [7]:
from tqdm import tqdm

In [8]:
for i in tqdm(range(len(df_stock)), desc='运行中', ncols=77):
    if df_stock.loc[i, 'SKU'].endswith('-NC'):
        df_stock = df_stock.drop(i)

运行中: 100%|█████████████████████| 269258/269258 [00:14<00:00, 18407.65it/s]


In [9]:
df_stock = df_stock.reset_index(drop=True)
df_stock['SKU'] = [df_stock.loc[i, 'SKU'][:-2] if df_stock.loc[i, 'SKU'].endswith('-N') else df_stock.loc[i, 'SKU'] for i in range(len(df_stock))]
df_stock = df_stock.groupby([df_stock['SKU']], as_index=False).sum()

df_stock

,SKU,当日库存
0,BPD8550086500+BRS31318,1
1,CAFP9183M0,1
2,CAWR41163,1
3,CS20230609,0
4,CS230530,0
...,...,...
82390,YHTWS58202-Y-6,0
82391,YHTWS5W1ZBA-Y-10,0
82392,YHTWS5W1ZBA-Y-8,0
82393,YHTWS802AA-Y-6,0


In [10]:
df_alternate['当日库存'] = 0
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='主料SKU', right_on='SKU', suffixes=('', '_0')).fillna(0)
print('stock - 主料SKU')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料1', right_on='SKU', suffixes=('', '_1')).fillna(0)
print('stock - 替换料1')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料2', right_on='SKU', suffixes=('', '_2')).fillna(0)
print('stock - 替换料2')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料3', right_on='SKU', suffixes=('', '_3')).fillna(0)
print('stock - 替换料3')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料4', right_on='SKU', suffixes=('', '_4')).fillna(0)
print('stock - 替换料4')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料5', right_on='SKU', suffixes=('', '_5')).fillna(0)
print('stock - 替换料5')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料6', right_on='SKU', suffixes=('', '_6')).fillna(0)
print('stock - 替换料6')

df_alternate['当日库存'] = df_alternate['当日库存_0'] + df_alternate['当日库存_1'] + df_alternate['当日库存_2'] + df_alternate['当日库存_3'] + df_alternate['当日库存_4'] + df_alternate['当日库存_5'] + df_alternate['当日库存_6']
df_alternate = df_alternate[['主料SKU',
                             '替换料1',
                             '替换料2',
                             '替换料3',
                             '替换料4',
                             '替换料5',
                             '替换料6',
                             '当日库存']]

df_alternate

stock - 主料SKU
stock - 替换料1
stock - 替换料2
stock - 替换料3
stock - 替换料4
stock - 替换料5
stock - 替换料6


,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,当日库存
0,USCA171876D-3,USCA171876-3,USCA171876,USCA171876D,,,,13.0
1,USGS00429LR,USGS00429,USGS00429T,,,,,52.0
2,USWL02211,USWL02211H,,,,,,39.0
3,USTC35518,USETC35518D,USETC35518,,,,,38.0
4,USTHA7040P,USTHA7040PX,,,,,,28.0
...,...,...,...,...,...,...,...,...
44798,USGS01664-2,USGS01664-2-C,,,,,,5.0
44799,USASC06160-C,USASC06160,,,,,,6.0
44800,USASC16160-C,USASC16160,,,,,,6.0
44801,USASC20303-C,USASC20303,,,,,,6.0


In [11]:
list_df = []
for i in ['主料SKU', '替换料1', '替换料2', '替换料3', '替换料4', '替换料5', '替换料6']:
    df_temp =  df_alternate[[i,
                             '当日库存']]
    df_temp.columns = ['SKU',
                       '当日库存']
    list_df.append(df_temp)

df_position = pd.concat(list_df)
df_position = df_position[df_position['SKU'] != ''].reset_index(drop=True)

df_position

,SKU,当日库存
0,USCA171876D-3,13.0
1,USGS00429LR,52.0
2,USWL02211,39.0
3,USTC35518,38.0
4,USTHA7040P,28.0
...,...,...
96342,USGS6310TNZB-C,0.0
96343,USIC2A366Z-C,2.0
96344,USIC04470I,0.0
96345,USIC02680I-6,33.0


## 2.2 MRP表库存

In [12]:
df_mrp = pd.read_excel(mrp,
                       header=1,
                       usecols='B').fillna(0)

df_mrp

,MRP SKU
0,USTCV17215HIE-2-C
1,USWP51040WH-C
2,USBDA04108-2-C
3,USFIK50300-C
4,USET87039H-C
...,...
60788,USDLA54516-C
60789,USEAF69000R-2-C
60790,USWR78870-C
60791,USAST50151-C


In [13]:
df_mrp = pd.merge(left=df_mrp, right=df_stock, how='left', left_on='MRP SKU', right_on='SKU').fillna(0)
df_mrp = pd.merge(left=df_mrp, right=df_position, how='left', left_on='MRP SKU', right_on='SKU', suffixes=('_stock', '_position')).fillna('')

df_mrp['库存'] = [df_mrp.loc[i, '当日库存_position'] if df_mrp.loc[i, '当日库存_position'] != '' else df_mrp.loc[i, '当日库存_stock'] for i in range(len(df_mrp))]
df_mrp = df_mrp[['MRP SKU',
                 '库存']]

df_mrp

,MRP SKU,库存
0,USTCV17215HIE-2-C,181.0
1,USWP51040WH-C,59.0
2,USBDA04108-2-C,172.0
3,USFIK50300-C,24.0
4,USET87039H-C,61.0
...,...,...
60788,USDLA54516-C,1.0
60789,USEAF69000R-2-C,2.0
60790,USWR78870-C,3.0
60791,USAST50151-C,2.0


# 3. 保存

In [14]:
import datetime

In [15]:
writer = pd.ExcelWriter('Lennon_整理好的库存表_' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx')
df_mrp.to_excel(writer, index=False)
writer.close()